In [5]:
%load_ext autoreload
%autoreload 2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("forces_only_f", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)


def prepare_x(x):
    objs = pad_array(list_to_wp_seq(x[:,obj_poses_indices],d=3),4,axis=-1) # no speed
    trajs = list_to_wp_seq(x[:,traj_indices],d=4)
    #   return np.concatenate([objs,trajs],axis = 1)
    return trajs[:,:-1,:]

test_dataset = tf.data.Dataset.from_tensor_slices((prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda
loading dataset:  forces_only_f ...done
raw X: (10000, 953) 	Y: (10000, 160)
filtered X: (10000, 953) 	Y: (10000, 160)
Train X: (7000, 953) 	Y: (7000, 160)
Test  X: (2000, 953) 	Y: (2000, 160)
Val   X: (1000, 953) 	Y: (1000, 160)


In [6]:
from src.TF4D_decoder_only import *

model_path = models_folder+"forces_onl/"
model_name = "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5"

model_file = model_path+model_name
model = load_model(model_file, delimiter="-")

# metrics = evaluate_model(model, x_t, y_t)
# models_metrics[model_tag] = metrics

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 16, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': True, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse'}
loading weights:  /home/arthur/local_data/models/forces_onl/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5


In [3]:
pred = model.predict(x_t)
pred_d = np.array(data)[indices_test]
for i,d in enumerate(pred_d):
    d["forces"] = pred[i]

63/63 [==============================] - 3s 17ms/step


In [4]:
%matplotlib qt
indices = np.random.choice(range(len(indices_test)), 3)

plt.close('all')
# pred_t = np.transpose(pred[:,:,:2],[0,2,1])
data_array = np.array(data)[indices_test[indices]]
show_data4D(data_array,plot_output=False)
# data_array = pred_d[indices]
# show_data4D(data_array,plot_output=False)

# show_data4D(data_array,plot_output=False,image_loader=mr.image_loader, color_traj=False, change_img_base=["/home/mirmi/Arthur/dataset/","/home/tum/data/image_dataset/"])
# show_data4D(data_sample,plot_output=False)#,image_loader=mr.image_loader)

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


In [147]:
%matplotlib qt

def interpolate_traj(wps,traj_n=40, offset=[0,0,0,0]):
    #create spline function
    f, u = interpolate.splprep([wps[:,0],wps[:,1],wps[:,2]], s=0)
    xint,yint,zint= interpolate.splev(np.linspace(0, 1, traj_n), f)

    tck,u = interpolate.splprep([np.linspace(0,1,len(wps[:,3])), wps[:,3]])
    velint_x, velint = interpolate.splev(np.linspace(0, 1, traj_n), tck)

    traj = np.stack([xint,yint,zint,velint],axis=1)+offset
    return traj


def norm_traj_and_objs(t, o, margin=0.45):
    pts_ = np.concatenate([o,t])

    vel = pts_[:,3:]
    pts = pts_[:,:3]

    vel_min = np.min(vel,axis = 0)
    vel_max = np.max(vel,axis = 0)
    vel_norm = np.max(np.abs(vel_max-vel_min))
    if vel_norm > 1e-10:
        vel = ((vel-(vel_max-vel_min)/2)/vel_norm)*(1-margin)

    else:
        vel = vel-vel_min

    pts_min = np.min(pts,axis = 0)
    pts_max = np.max(pts,axis = 0)
    pts_norm = np.max(np.abs(pts_max-pts_min))

    # pts  = ((pts-pts_min)/pts_norm)*(1-margin)+margin/2-0.5
    pts  = ((pts-(pts_max-pts_min)/2)/pts_norm)*(1-margin)

    pts_new= np.concatenate([pts,vel],axis=-1)
    o_new = pts_new[:o.shape[0],:]
    t_new = pts_new[o.shape[0]:,:]

    return t_new, o_new, [pts_norm, (pts_max-pts_min)/2,vel_norm, (vel_max-vel_min)/2, margin]

def rescale(pts_, factor_list):

    vel = pts_[:,3:]
    pts = pts_[:,:3]

    pts_norm, pts_avr,vel_norm, vel_avr, margin = factor_list
    pts = pts/(1-margin)*pts_norm+pts_avr
    vel = vel/(1-margin)*vel_norm+vel_avr
    pts_new= np.concatenate([pts,vel],axis=-1)

    return pts_new

def modify_traj(mr, traj, obj_poses, text, obj_names,obj_poses_offset, show=False):

    images = None

    print("---------------------------------------")
    print(text)
    # print(traj.shape, obj_poses.shape, text, obj_names,obj_poses_offset.shape)

    traj_, obj_poses_, factor_list = norm_traj_and_objs(traj, obj_poses)
    traj_int = interpolate_traj(traj_,traj_n=traj_n)
    
    obj_poses_ = obj_poses_[:,:3]

    d = np2data(traj_int, obj_names, obj_poses_, text)[0]

    pred, traj_in = mr.apply_interaction(model, d, text,  label=False, images=images, dec_only=True)

    # %matplotlib qt
    if show:
        data_array = np.array([d])
        # show_data4D(data_array, forces=pred, color_traj=False)
        # print(traj_in.shape)
        objs  = {}
        obj_names = np.asarray(d["obj_names"])
        obj_pt = np.asarray(d["obj_poses"])
        for x,y,z,name in zip(obj_pt[:,0],obj_pt[:,1],obj_pt[:,2],obj_names):
            objs[name] = {"value":{"obj_p":[x,y,z]}}
        
        # plot_samples(text,traj_[-2000:-1000],[], images=[], fig=None,objs=objs, colors = ["#02b322","#0c45c2","#C2450C"],alpha=[0.9,0.9],linewidth=1.0,
        #         plot_voxels= False, color_traj = False, map_cost_f=None, labels=[], plot_speed=False, show=True, forces=None)

        plot_samples(text,traj_in[0],[], images=[], fig=None,objs=objs, colors = ["#02b322","#0c45c2","#C2450C"],alpha=[0.9,0.9],linewidth=1.0,
                plot_voxels= False, color_traj = False, map_cost_f=None, labels=[], plot_speed=False, show=True, forces=pred[0])
    # 
    traj_new = rescale(pred[0], factor_list)
    
    traj_new = interpolate_traj(traj_new,traj_n=np.array(traj).shape[0])
    # publish_simple_traj(traj_new,obj_poses+obj_poses_offset,new_traj_pub)
    # publish_simple_traj(traj,obj_poses+obj_poses_offset,traj_pub)

    return traj_new,d

from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS

forces_data_folder = "/home/arthur/local_data/forces_data/"
forces_data_folder_new = "/home/arthur/local_data/forces_data_new/"
text = "scoope deeper"
text = "scoope a lot more down"
# text = "press a lot stronger down"
# text = "press stronger down"

# text = "scoope much deeper down"

obj_poses = np.array([[0,0,-0.5,0],[0,0.4,-0.5,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
obj_poses_offset = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
obj_names = ["sand","table"]
plt.close()
for force_file in os.listdir(forces_data_folder)[2:]:
    if not "9" in force_file:
        continue
    # try:
    print(force_file)
    force_file_file = os.path.join(forces_data_folder,force_file)
    traj=[]
    with open(force_file_file) as f:
        lines = f.readlines()
    for i,l in enumerate(lines):
        traj.append([float(c) for c in l[:-1].split(",")]+[0])

    traj_new,d = modify_traj(mr, traj, obj_poses, text, obj_names,obj_poses_offset, show=True)
    
    force_file_file_new = os.path.join(forces_data_folder_new,force_file)
    with open(force_file_file_new, 'w') as f:
        for wp in traj_new:
            line = str(wp[0])+" ,"+str(wp[1])+" ,"+str(wp[2])
            f.write(line)
            f.write('\n')

    # except:
    #     pass
    # plt.close() 
    # fig = plt.figure(figsize=(10,13))
    # plt.plot(range(len(traj_new[:,0])),traj_new[:,2])
    # plt.plot(range(len(traj_new[:,0])),traj_new[:,1])
    # traj
    break


Cart9.txt
---------------------------------------
scoope a lot more down
input (40, 4)
DONE - computing textual embeddings (1, 768)


1it [00:00, 147.62it/s]

DONE - computing similarity vectors 
DONE - concatenating 
X shape (1, 953)
1/1 [==============================] - 0s 22ms/step
